In [2]:
import pandas as pd
import numpy as np
import re
import string

# A function for cleaning texts

 Persian stop words and a list of all English chars

In [3]:
# list of persian stop words
persian_stop_words = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/persian_stop_words.txt")
persian_stop_words_list = persian_stop_words["کلمات ایستای فارسی"].tolist()
# lsit of all English letters
english_letters_list = list(string.ascii_letters)

This function removes all emogies from input text

In [4]:
def deEmojify(text):
  regrex_pattern = re.compile(pattern = "["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          "]+", flags = re.UNICODE)
  return regrex_pattern.sub(r'',text)

This function removes all punctuations, English letters, persian stop words, and digits from an input text and returns a clean text

In [5]:
def preprocess_text(text, english_letters_list, persian_stop_words_list):
  
  # remove punctuations from string(!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~)
  text = text.translate(str.maketrans('', '', string.punctuation))

  # remove all English letters and all digits from string
  text = ''.join(char for char in text if (char not in english_letters_list and not char.isdigit()))

  # removing all emogies from string
  text = deEmojify(text)
  
  # convert from string to list
  lst_text = text.split()

  # remove Persian stopwords
  if persian_stop_words_list is not None:
    lst_text = [word for word in lst_text if word not in persian_stop_words_list]
  text = " ".join(lst_text)

  return text

## Reading Data and cleaning it

Reading the crawled json file of digikala products info and turning it into a cleaned numpy array of unique product names containing 2527 products

In [6]:
products_info = pd.read_json("/content/drive/MyDrive/YektaNet_Interview/crawled_digikala_products_info.jl", lines=True)
products = products_info.drop(columns=["id", "c", "r", "adjs", "cmts"])
products.rename(columns={"t": "product_name"}, inplace=True)
products.dropna(inplace=True)

# Cleaning all data from redundant letters
products["product_name_clean"] = products["product_name"].apply(lambda x: 
  preprocess_text(x, english_letters_list, persian_stop_words_list))

unique_products = products["product_name_clean"].unique()
unique_products.shape

(2527,)

In [ ]:
all_unique_products = products.drop_duplicates(subset=['product_name_clean']).reset_index().drop(columns=["index"])
all_unique_products.to_csv('/content/drive/MyDrive/YektaNet_Interview/all_unique_products.csv', index=False) 
all_unique_products

,product_name,product_name_clean
0,دوربین دیجیتال کانن مدل EOS 850D به همراه لنز ...,دوربین دیجیتال کانن مدل همراه لنز میلی متر
1,مینی فرز ادون مدل AG115-1002T,مینی فرز ادون مدل
2,دوربین فیلم برداری ورزشی گوپرو مدل HERO 9 Blac...,دوربین فیلم ورزشی گوپرو مدل همراه لوازم جانبی
3,دوربین فیلم برداری خودرو کراس تور مدل CR300,دوربین فیلم خودرو کراس تور مدل
4,کمپرسور هوا مدل M2020,کمپرسور هوا مدل
...,...,...
2522,سرویس خواب 3 تکه نوزادی ببتو مدل Pat22Rabbit,سرویس خواب تکه نوزادی ببتو مدل
2523,سرویس خواب 5 تکه کودک مدل خرس های مهربون,سرویس خواب تکه کودک مدل خرس مهربون
2524,سرویس خواب چهار تکه کودک طرح جزیره,سرویس خواب تکه کودک طرح جزیره
2525,سرویس خواب 9 تکه کودک کیدبو مدل Twinkle-star,سرویس خواب تکه کودک کیدبو مدل


Reading the web.csv data and droping all columns but the fulltext. We also keep the categories columns to check the precision of our predictions. Then we apply the preprocess_text function to clean all the fulltext column

In [ ]:
web_data = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/web.csv", encoding='utf-8')
# define proper columns for the dataset to make it more understandable
web_data.columns = ['id', 'website', 'fulltext', 'txt1', 'txt2', 'txt3', 'txt4', 'keywords', 'un1', 'un2', 'un3', 'un4', 'un5', 'un6', 'categories'] 
# droping all columns except for fulltext and categories
web_data = web_data.drop(columns=['id', 'website', 'txt1', 'txt2', 'txt3', 'txt4', 'keywords', 'un1', 'un2', 'un3', 'un4', 'un5', 'un6'])
# removing rows containing error
web_data = web_data[~web_data.fulltext.str.contains("Error")]
web_data = web_data.reset_index().drop(columns=["index"])
# Cleaning all fulltext columns from redundant letters
web_data["fulltext_clean"] = web_data["fulltext"].apply(lambda x: 
  preprocess_text(x, english_letters_list, persian_stop_words_list))

dropping duplicate rows in fulltext_clean column and reseting indexes

In [ ]:
web_data = web_data.drop_duplicates(subset=['fulltext_clean']).reset_index().drop(columns=["index"])
web_data

,fulltext,categories,fulltext_clean
0,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | ت...,"{""20"": {""probability"": ""0.2120969927416129"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم تولدت م...
1,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | چ...,"{""9"": {""probability"": ""0.15588014951828624"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم شیر شیر...
2,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | ن...,"{""30"": {""probability"": ""0.5822832917369573"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم نتیجه آ...
3,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | پ...,"{""20"": {""probability"": ""0.3405926198476455"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم پای امی...
4,من تنها کفشدوزکی هستم که خدارا دید | خرنوب عسل...,"{""22"": {""probability"": ""0.26305950139135476"", ...",کفشدوزکی خدارا دید خرنوب عسل ودارچین تحرکی اسپ...
...,...,...,...
11181,روزهای بد امیری در سوپرلیگ ترکیه - یک ورزش به ...,"{""13"": {""probability"": ""0.517595361945674"", ""s...",امیری سوپرلیگ ترکیه ورزش نقل ، هفته رقابت‌های ...
11182,تساوی ترابزون اسپور با امیری و حسینی - یک ورزش...,"{""13"": {""probability"": ""0.7573625064373598"", ""...",تساوی ترابزون اسپور امیری حسینی ورزش ترابزون ا...
11183,شکست سنگین ترابزون با حسینی - یک ورزش ترابزون ...,"{""13"": {""probability"": ""0.6072557407472986"", ""...",شکست سنگین ترابزون حسینی ورزش ترابزون اسپور جم...
11184,یک تیم لالیگایی به دنبال حسینی - یک ورزش به نق...,"{""13"": {""probability"": ""0.4466869902891258"", ""...",تیم لالیگایی دنبال حسینی ورزش نقل روزنامه گازت...


In [ ]:
# compression_opts = dict(method='zip', archive_name='web_data_clean.csv')  
# web_data.to_csv('/content/drive/MyDrive/YektaNet_Interview/web_clean.zip', index=False, compression=compression_opts) 

In [ ]:
# import zipfile
# with zipfile.ZipFile("/content/drive/MyDrive/YektaNet_Interview/web_clean.zip", 'r') as zip_ref:
#   zip_ref.extractall("/content/drive/MyDrive/YektaNet_Interview/")

loading wiki.txt dataset and applying all the mentioned things on it to make it clean 

In [ ]:
wiki_data = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/wiki.txt", encoding='utf-8')
wiki_data.columns = ['fulltext'] 
# removing rows containing error
wiki_data = wiki_data[~wiki_data.fulltext.str.contains("Error")]
wiki_data = wiki_data.reset_index().drop(columns=["index"])
# Cleaning all fulltext columns from redundant letters
wiki_data["fulltext_clean"] = wiki_data["fulltext"].apply(lambda x: 
  preprocess_text(x, english_letters_list, persian_stop_words_list))
# droping duplicate rows and reseting indexes
wiki_data = wiki_data.drop_duplicates(subset=['fulltext_clean']).reset_index().drop(columns=["index"])

In [ ]:
# compression_opts = dict(method='zip', archive_name='wiki_data_clean.csv')  
# wiki_data.to_csv('/content/drive/MyDrive/YektaNet_Interview/wiki_clean.zip', index=False, compression=compression_opts) 
wiki_data

,fulltext,fulltext_clean
0,سالنامه دفتر یا کتابی است دست نویس یا چاپ شده ...,سالنامه دفتر کتابی دست نویس چاپ درمورد رویداده...
1,اط لاع یا آگاهش در کوتاه ترین تعریف داده های پ...,اط لاع آگاهش کوتاه تعریف پردازش مواد خام بالقو...
2,این نگاره نمونه ای از یک اثر ویرایش شده است که...,نگاره نمونه اثر ویرایش تصویرهای گوناگونی اجازه...
3,ویکی معمولا به انواعی از وب گاه ها گفته می شود...,ویکی انواعی وب بازدیدکنندگانش اوقات نیاز ثبت و...
4,نرم افزارهای مشارکت گرا گروه افزار یا ابزار گر...,نرم افزارهای مشارکت گرا گروه افزار ابزار نرم ا...
...,...,...
19994,پرچم آمریکا در جزایر کوچک حاشیه ای ایالات متحد...,پرچم آمریکا جزایر کوچک حاشیه ایالات متحده رسمی...
19995,پرونده بندانگشتی نقشه بالا طرح نهایی جزایر شیخ...,پرونده بندانگشتی نقشه طرح نهایی جزایر شیخ نشین...
19996,شهر هارتفورد پایتخت ایالتی کنتیکت در کشور آمری...,شهر هارتفورد پایتخت ایالتی کنتیکت کشور آمریکا ...
19997,دامنه اختصاص داده شده به شوروی سابق است امکان...,دامنه اختصاص شوروی امکان ثبت دامنه منابع پیوند...


# Loading all cleaned datasets

In [8]:
all_unique_products = pd.read_csv('/content/drive/MyDrive/YektaNet_Interview/all_unique_products.csv') 
web_data = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/web_data_clean.csv")
wiki_data = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/wiki_data_clean.csv")

## Let's make a Recommender system!

In [ ]:
!pip install transformers

In [10]:
import transformers

Importing the pretrained bert network which is fine tuned by persian datasets by HooshvareLab called ParsBERT

In [11]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel, AutoModel

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = TFAutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Some layers from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


This function breaks a large string input into several small strings to make it possible for the bert model to vectorize it

In [12]:
def splitter(long_string, window_length):

  all_subs = []
  splitted_string = long_string.split()
  i = 0
  while i + window_length < len(splitted_string):
    substring = splitted_string[i: i + window_length]
    i = i + window_length
    all_subs.append(' '.join(substring))
    if i + window_length > len(splitted_string):
      substring = splitted_string[i: ]
      all_subs.append(' '.join(substring))
  return all_subs

This function tokenizes each input string and then embeds it using bert model. if bert model is unable to handle the whole string in the first place, we break it using splitter and embed each part of string using bert

In [13]:
def utils_bert_embedding(txt, tokenizer, model, window_length):

  all_embeds_string = []
  try:
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = model(idx)
    X = np.array(embedding[0][0][1:-1])
    all_embeds_string.append(X.mean(0))
  except:
    sub_strings = splitter(txt, window_length)
    for i in range(len(sub_strings)):
      idx = tokenizer.encode(sub_strings[i])
      idx = np.array(idx)[None,:]  
      embedding = model(idx)
      X = np.array(embedding[0][0][1:-1])
      all_embeds_string.append(X.mean(0))

  return all_embeds_string

Now we use the utils_bert_embedding to vectorize the first 2000 rows of cleaned web_data dataset. Each row will have at least one vector with size of 768, and if it is too big, it will have 2 or more vectors with size of 768, for each part of it

In [ ]:
## create list of web_data vector
all_embeds_web_data = []
window_length = 100
cnt = 0
for index, row in web_data.loc[0:2000].iterrows():
  all_embeds_web_data.append(utils_bert_embedding(row["fulltext_clean"], tokenizer, model, window_length))
  cnt += 1
  if cnt % 100 == 0:
    print(cnt)

In [ ]:
first_2000_embeds_web_data = np.asanyarray(all_embeds_web_data)
np.save("/content/drive/MyDrive/YektaNet_Interview/first_2000_embeds_web_data.npy", first_2000_embeds_web_data)

Now we use the utils_bert_embedding to vectorize the first 1000 rows of cleaned wiki_data dataset. Each row will have at least one vector with size of 768, and if it is too big, it will have 2 or more vectors with size of 768, for each part of it

In [ ]:
## create list of wiki_data vector
all_embeds_wiki_data = []
window_length = 100
cnt = 0
for index, row in wiki_data.loc[0:1000].iterrows():
  all_embeds_wiki_data.append(utils_bert_embedding(row["fulltext_clean"], tokenizer, model, window_length))
  cnt += 1
  if cnt % 100 == 0:
    print(cnt)

In [ ]:
first_1000_embeds_wiki_data = np.asanyarray(all_embeds_wiki_data)
np.save("/content/drive/MyDrive/YektaNet_Interview/first_1000_embeds_wiki_data.npy", first_1000_embeds_wiki_data)

Now we use the utils_bert_embedding to vectorize all 2527 cleaned product names:

In [ ]:
## create list of unique product names vector
all_products_embeds = []
window_length = 100
cnt = 0
for product in unique_products:
  all_products_embeds.append(utils_bert_embedding(product, tokenizer, model, window_length))
  cnt += 1
  if cnt % 500 == 0:
    print(cnt)

In [ ]:
all_unique_products_embeds = np.asanyarray(all_products_embeds)
np.save("/content/drive/MyDrive/YektaNet_Interview/all_unique_products_embeds.npy", all_unique_products_embeds)

## Now we load the vectorized arrays of web_data, wiki_data, and products names which we made and saved as numpy arrays

In [14]:
from numpy import load
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [15]:
first_2000_web_data_vectors = np.load("/content/drive/MyDrive/YektaNet_Interview/first_2000_embeds_web_data.npy")
first_1000_wiki_data_vectors_ = np.load("/content/drive/MyDrive/YektaNet_Interview/first_1000_embeds_wiki_data.npy")
all_unique_products_vectors = np.load("/content/drive/MyDrive/YektaNet_Interview/all_unique_products_embeds.npy")

# Let's predict best products for each data

The metric we use to find out the similarity between each data vector and each product is cosine similarity which could be calculated as below

In [16]:
import scipy
from scipy import spatial
cosine_similarity = 1 - spatial.distance.cosine(first_2000_web_data_vectors[3][0], all_unique_products_vectors[3][0])
cosine_similarity

0.45193254947662354

Now we write functions to calculate the cosine similarity between each vectorized data in a dataset and products. Then we choose k products with most cosine similarity with the input data vector and recommend them

In [17]:
from collections import OrderedDict

def find_k_largest_indices(a, k):
  idx = np.argpartition(a.ravel(),a.size-k)[-k:]
  return np.column_stack(np.unravel_index(idx, a.shape))

def find_k_most_similar_products_from_indices(index_list):
  k_most_similar_products = []
  for i in range(len(index_list)):
    index = index_list[i][0]
    k_most_similar_products.append(unique_products[index])
  return k_most_similar_products

def find_k_products_with_most_cos_similarity_with_data(vec, products_vecs, k):
  all_cos_sims = []
  for label_vec in products_vecs:
    cosine_similarity = 1 - spatial.distance.cosine(vec[0], label_vec)
    all_cos_sims.append(cosine_similarity)
  k_largest_indices = find_k_largest_indices(np.asanyarray(all_cos_sims), k)
  return find_k_most_similar_products_from_indices(k_largest_indices)

def recommender(data_vecs, products_vecs, k):
  all_predicted_labels = []
  for vec in data_vecs:
    if len(vec) == 1:
      k_most_similar_products = find_k_products_with_most_cos_similarity_with_data(vec, products_vecs, k)
    else:
      most_similars_lists = []
      for vector in vec:
        k_most_similar_products = find_k_products_with_most_cos_similarity_with_data(vector, products_vecs, k)
        most_similars_lists.append(k_most_similar_products)
      
      k_most_similar_products = list(OrderedDict.fromkeys(sum(most_similars_lists, [])))
    
    all_predicted_labels.append(k_most_similar_products)
  return all_predicted_labels

# Let's Test

Predicting related products for the first 2000 web_data rows with k = 5

In [ ]:
recommended_products_for_web_data = recommender(first_2000_web_data_vectors, all_unique_products_vectors, 5)

In [ ]:
first_2000_web_data = web_data[0:2001]
first_2000_web_data["recommended_products"] = recommended_products_for_web_data

In [ ]:
compression_opts = dict(method='zip', archive_name='first_2000_web_data_clean_with_recommends.csv')  
first_2000_web_data.to_csv('/content/drive/MyDrive/YektaNet_Interview/first_2000_web_data_clean_with_recommends.zip', index=False, compression=compression_opts) 

In [ ]:
# import zipfile
# with zipfile.ZipFile("/content/drive/MyDrive/YektaNet_Interview/first_2000_web_data_clean_with_recommends.zip", 'r') as zip_ref:
#     zip_ref.extractall("/content/drive/MyDrive/YektaNet_Interview/")

loading output data for first 2000 web data

In [18]:
first_2000_web_data_with_recommends = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/first_2000_web_data_clean_with_recommends.csv")

In [ ]:
first_2000_web_data_with_recommends

,fulltext,categories,fulltext_clean,recommended_products
0,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | ت...,"{""20"": {""probability"": ""0.2120969927416129"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم تولدت م...,"['سشوارحرفه پرنسلی مدل', 'ماشین اصلاح بدن رمین..."
1,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | چ...,"{""9"": {""probability"": ""0.15588014951828624"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم شیر شیر...,"['ست ماشین اصلاح موی بدن رمینگتون مدل', 'ماشین..."
2,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | ن...,"{""30"": {""probability"": ""0.5822832917369573"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم نتیجه آ...,"['ماشین اصلاح رمینگتون', 'ماشین اصلاح بدن رمین..."
3,اميرحسین جون و امیرمحمد جون و امیرعلی جونم | پ...,"{""20"": {""probability"": ""0.3405926198476455"", ""...",اميرحسین جون امیرمحمد جون امیرعلی جونم پای امی...,"['ماشین اصلاح بدن رمینگتون مدل', 'ست ماشین اصل..."
4,من تنها کفشدوزکی هستم که خدارا دید | خرنوب عسل...,"{""22"": {""probability"": ""0.26305950139135476"", ...",کفشدوزکی خدارا دید خرنوب عسل ودارچین تحرکی اسپ...,"['شامپو طراوت شون حجم میلی لیتر', 'نرم موی کود..."
...,...,...,...,...
1996,خبرگزاری فارس - مصدومیت پدر و پسر 15ساله و خود...,"{""15"": {""probability"": ""0.22950841099597985"", ...",خبرگزاری فارس مصدومیت پدر پسر ساله خودزنی اسید...,"['سرویس خواب تکه نوزاد صبااو مدل', 'سرویس خواب..."
1997,خبرگزاری فارس - می‌خواهند لایحه الحاق به کنوان...,"{""15"": {""probability"": ""0.5902321100889905"", ""...",خبرگزاری فارس می‌خواهند لایحه الحاق کنوانسیون ...,"['روغن گیربکس خودرو پروفی کار مدل حجم لیتر', '..."
1998,خبرگزاری فارس - سیاست های آمریکا در منطقه شکست...,"{""15"": {""probability"": ""0.5858267245574352"", ""...",خبرگزاری فارس سیاست آمریکا منطقه شکست خورده وج...,"['وبالشی کودک طرح فروزن مدل', 'شامپو بدن نوزاد..."
1999,خبرگزاری فارس - مشایی به حکم دادگاه اعتراض کرد...,"{""15"": {""probability"": ""0.44981806069913133"", ...",خبرگزاری فارس مشایی حکم دادگاه اعتراض گزارش خب...,"['مکمل سوخت خودرو لیکومولی مدل حجم میلی لیتر',..."


In [122]:
first_2000_web_data_with_recommends["fulltext"][33]

'نی نی وبلاگ | درسا مروارید قلب مامان وباباش عاشقی این مطلب رمز دار است نویسنده : عاشق درسا 15:57 15 تير 1393 287 0 0 ادامه مطلب بهترین عید عشقم تو کنارمی.نازم حست میکنم، نازت میکنم شیرینم میبوسمت،توبامایی امسال کنارمی نازم.الحمدالله شاکرتم خدا به خاطر خنده های درسا به خاطر چهار دست وپارفتن درسا،به خاطر شیرین زبونی هاش، به خاطر خنده های درسا که شادی فراوون را به ما هدیه کرده. امسال عید بوی گلهای نسترن میده حتی ماهیهای تنگ بلورم امسال رقصان ترند.ای خدای زیباییها امسال را بر ما آسان گردان وزیباییهات را بر ما هدیه کن. خدای مهربان خانواده ام رابه تو می سپارم وسلامتی وشادابی را برای تمام دوستانم از تو مهربان خواستارم. الهی آمین. &nb... نویسنده : عاشق درسا 19:40 1 فروردين 1393 305 6 0 ادامه مطلب ,پرستار حضورتان اشاره ایست زینب گونه، شاید دردمندی به دستان تو دخیل بسته باشه .......... دستان شفا بخش روزت مبارک.روز پرستار به تمام پرستاران عزیز مبارک.اميدوارم خداوند به خاطر این شب بیداریها قسمتی از بهشت نصیبمان کند. نویسنده : عاشق درسا 19:33 15 اسفند 1392 311 0 7 ادامه مطلب مژده مژده عکسهای 5 ماه

In [123]:
print(first_2000_web_data_with_recommends["recommended_products"][33])

['روبالشی کودک طرح فروزن مدل', 'بالش کودک طرح دشت زیبا', 'ست ماشین اصلاح بدن رمینگتون', 'روبالشی کودک طرح مردعنکبوتی مدل', 'فيله ماهي گاليت ممتاز شارین گرم']


Predicting related products for the first 1000 wiki_data rows with k = 5

In [ ]:
recommended_products_for_wiki_data = recommender(first_1000_wiki_data_vectors_, all_unique_products_vectors, 5)

In [ ]:
first_1000_wiki_data = wiki_data[0:1001]
first_1000_wiki_data["recommended_products"] = recommended_products_for_wiki_data

In [ ]:
compression_opts = dict(method='zip', archive_name='first_1000_wiki_data_clean_with_recommends.csv')  
first_1000_wiki_data.to_csv('/content/drive/MyDrive/YektaNet_Interview/first_1000_wiki_data_clean_with_recommends.zip', index=False, compression=compression_opts) 

In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/YektaNet_Interview/first_1000_wiki_data_clean_with_recommends.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/YektaNet_Interview/")

loading out put for first 1000 wiki data

In [124]:
first_1000_wiki_data_with_recommends = pd.read_csv("/content/drive/MyDrive/YektaNet_Interview/first_1000_wiki_data_clean_with_recommends.csv")

In [ ]:
first_1000_wiki_data_with_recommends

,fulltext,fulltext_clean,recommended_products
0,سالنامه دفتر یا کتابی است دست نویس یا چاپ شده ...,سالنامه دفتر کتابی دست نویس چاپ درمورد رویداده...,['ماشین اصلاح مدل همراه ست شانه عددی ماشین اصل...
1,اط لاع یا آگاهش در کوتاه ترین تعریف داده های پ...,اط لاع آگاهش کوتاه تعریف پردازش مواد خام بالقو...,['ست شانه ماشین اصلاح مدل بسته عددی همراه روغن...
2,این نگاره نمونه ای از یک اثر ویرایش شده است که...,نگاره نمونه اثر ویرایش تصویرهای گوناگونی اجازه...,['کاور سیلیکونی پروتکتیو کیس پروتکتیو کیس مدل ...
3,ویکی معمولا به انواعی از وب گاه ها گفته می شود...,ویکی انواعی وب بازدیدکنندگانش اوقات نیاز ثبت و...,"['روغن موتور خودرو پروفی کار مدل حجم لیتر', 'م..."
4,نرم افزارهای مشارکت گرا گروه افزار یا ابزار گر...,نرم افزارهای مشارکت گرا گروه افزار ابزار نرم ا...,['دوربین فیلم ورزشی گوپرو مدل همراه کیف لوازم ...
...,...,...,...
996,آدانا نام یکی از استان های ترکیه است که مرکز آ...,آدانا استان ترکیه مرکز شهری جغرافیا استان ساحل...,"['گچ مو هات هیوز کد مجموعه عددی', 'مکمل سوخت خ..."
997,بالیکسیر نام یکی از استان های ترکیه است که مرک...,بالیکسیر استان ترکیه مرکز شهری یونانی منطقه پا...,"['فيله ماهي گاليت ممتاز شارین گرم', 'مسواک دنت..."
998,استان ختای یا حاتای یا هاتای نام یکی از استان ...,استان ختای حاتای هاتای استان ترکیه مرکزیت شهر ...,"['نرم موی کودک الوینا مدل حجم میل لیتر', 'سرم ..."
999,افیون قره حصار یکی از شهرهای ترکیه است این شهر...,افیون قره حصار شهرهای ترکیه شهر مرکز استان افی...,['شامپو مو استم سل مدل موهای خشک آسیب حجم میلی...


In [157]:
first_1000_wiki_data_with_recommends["fulltext_clean"][333]

'دیرانه صنعت حمل نقل خسارت دیرکرد تخلیه وسیله بارکش آهن خسارت دیرکرد تخلیه واگن آهن مقصد آهن مبدأ پرداخت منابع فرهنگستان زبان فارسی رده بارگنج رده ترابری رده حقوق تجارت رده قانون دریایی رده واژگان حقوقی'

In [158]:
print(first_1000_wiki_data_with_recommends["recommended_products"][333])

['مکمل سوخت خودرو فولکس واگن مدل حجم میلی لیتر', 'روغن گیربکس خودرو پروفی کار مدل حجم لیتر', 'شانه ماشین اصلاح مدل مجموعه عددی', 'روغن موتور خودرو پروفی کار مدل حجم لیتر', 'مکمل روغن موتور خودرو لوکسیت مدل حجم میلی لیتر مجموعه عددی']


# Predicting most related products for one input string by a given threshold for cosine similarity

Here we write a function which gets an input string, a list of all product names, and a threshold for cosine similarity. First, we transform the input string into one or more vectors with size of 768(Using pretrained PARSBERT model), and find the cosine similarity of this vector with all product vectors. Finally, we return the products which their cosine similarity with the input vector is more than the given threshold

In [ ]:
def utils_bert_embedding_for_one_txt(txt, tokenizer, model):

  idx = tokenizer.encode(txt)
  idx = np.array(idx)[None,:]  
  embedding = model(idx)
  X = np.array(embedding[0][0][1:-1])
  return X.mean(0)

def find_products_with_cos_similarity_more_than_threshold(vectorized_txt, threshold, all_unique_products_vectors):

  products_with_cos_similarity_more_than_threshold = []
  for i in range(len(all_unique_products_vectors)):
    cosine_similarity = 1 - spatial.distance.cosine(vectorized_txt, all_unique_products_vectors[i][0])
    if cosine_similarity >= threshold:
      products_with_cos_similarity_more_than_threshold.append(unique_products[i])
  return products_with_cos_similarity_more_than_threshold


def recommender_with_threshold(txt, all_unique_products_vectors, threshold, tokenizer, model):
  
  vectorized_txt = utils_bert_embedding_for_one_txt(txt, tokenizer, model)
  recommended_products = find_products_with_cos_similarity_more_than_threshold(vectorized_txt, threshold, all_unique_products_vectors)
  return recommended_products

Now, we test this method

In [ ]:
txt = wiki_data["fulltext_clean"][0]
threshold = 0.65
recommender_with_threshold(txt, all_unique_products_vectors, threshold, tokenizer, model)

['کاغذ یادداشت طرح کارهای بسته عددی', 'تقویم رومیزی مستر راد سال مدل کد']

In [ ]:
wiki_data["fulltext_clean"][0]

'سالنامه دفتر کتابی دست نویس چاپ درمورد رویدادهای اتفاق افتاده سال اطلاعاتی سالنامه موضوعات مختلفی جمله اقتصادی سیاسی اجتماعی پوشش موضوع بپردازند سالنامه شامل فهرست مناسبت بنا تصمیماتی اتخاذ یاد خاطره حوادث اتفاقات انسان زنده نگه نوشتن سالنامه مواردی حقوقی بازرگانی انبارداری تولیدی جنبه قانونی انواع سالنامه مختلفی سال چاپ سالنامه هنرمندان سالنامه تئاتر سالنامه جاودانه ایران برد سالنامه تقویم تقویمی بطن اندازه سالنامه سررسید اندازه کوچک اندازه معمول سررسید سالنامه قطع کتاب سلطانی رحلی وزیری رقعی پالتویی خشتی جیبی منابع دکتر مسعود ارشادی سررسید سالنامه رده آکادمی رویدادها رده کتاب رده کتاب پایه'

## A recommender by simple search

Here, we write a function which gives a text as input and simply compare its words with all products names in the list. If there is any common word between them, the product will be recommended 

In [ ]:
def check_number_of_common_words(first_txt_list, second_txt_list):

  number_of_common_words_between_two_texts = 0
  for word in first_txt_list:
    if word in second_txt_list:
      number_of_common_words_between_two_texts += 1
  return number_of_common_words_between_two_texts

def recommend_by_search_with_threshold(txt, unique_products, threshold):
  recommended_products = []
  splitted_txt_list = txt.split()
  for product in unique_products:
    splitted_product_list = product.split()
    if check_number_of_common_words(splitted_txt_list, splitted_product_list) >= threshold:
      recommended_products.append(product)
  return recommended_products

In [ ]:
web_data["fulltext_clean"][1200]

'خبرگزاری فارس اختلاف قیمت خودرو کارخانه بازار حباب سعید مؤتمنی گفت\u200cوگو خبرنگار اقتصادی خبرگزاری فارس وضعیت معامله خودرو بازار نوسانات قیمت ارز منجر کاهش معاملات رئیس اتحادیه صنف نمایشگاه داران فروشندگان خودرو بیان خرید خودرو دقت بدانند فاصله قیمت خودرو کارخانه بازار تومان است، اختلاف قیمت حبابی پاسخ سوال فروش خودرو خودروسازان تاثیری بازار خودرو است، فروش\u200cها کوتاه بود، بازار خورو تاثیر می\u200cگذاشت می\u200cتوانست قیمت\u200cها کاهش دهد، حالی تحویل\u200c خودروهای فروش مربوط سال انتهای پیامب'

Lets test


In [ ]:
txt = web_data["fulltext_clean"][1200]
threshold = 1
recommend_by_search_with_threshold(txt, unique_products, threshold)

['دوربین فیلم خودرو کراس تور مدل',
 'اسپری انژکتور خودرو هافمن مدل حجم میلی لیتر',
 'روغن موتور خودرو ولوولين مدل حجم ميلی ليتر',
 'جک هیدرولیک خودرو کنزاکس مدل',
 'جک هیدرولیک خودرو نووا مدل',
 'جک هیدرولیک خودرو زبرا مدل',
 'جک هیدرولیک خودرو اینهل مدل',
 'کمپرسور خودرو مدل',
 'جک خودرو مدل',
 'کمپرسور هوا فندکی خودرو کنزاکس مدل',
 'جک خودرو آیرون مکس مدل',
 'جک خودرو کد',
 'جک هیدرولیک خودرو آروا مدل',
 'جک هیدرولیک خودرو وینکس مدل',
 'جک هیدرولیک خودرو آسا مدل',
 'جک خودرو مدل همراه دسته جک',
 'جک خودرو واکتا کد',
 'جک خودرو آروا مدل',
 'جک هیدرولیک خودرو رتتا مدل',
 'جک خودرو مدل مناسب خودروهای شاسی بلند',
 'جک خودرو رونیکس مدل',
 'خرک خودرو جکیران مدل',
 'جک خودرو تی مدل',
 'کمپرسور هوای خودرو ایکس وان مدل',
 'روغن موتور خودرو بوش مدل حجم میلی لیتر',
 'مکمل روغن موتور خودرو لیکومولی کد حجم میلی لیتر',
 'مکمل سوخت خودرو دیرگون مدل حجم میلی لیتر',
 'روغن ترمز خودرو هراز مدل حجم میلی\u200c لیتر',
 'ضد یخ خودرو پروفی کار مدل کیلوگرم',
 'جک خودرو کنزاکس مدل',
 'روغن ترمز خودرو آلتیما 